ModuleNotFoundError: No module named 'bert_score'

In [2]:
import pyasn1_modules.rfc5084
!pip install bert_score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 42.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 MB 41.4 MB/s eta 0:00:0000:0100:01
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 43.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.9/410.9 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.2/417.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.2/536.2 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typi

In [36]:
METHODS = ["RAG", "RAG+github"]
MODELS = ["gpt-3.5-turbo", "gpt-4", "nous-hermes-13b"]


In [3]:
import pandas as pd
import yaml
import glob

def get_method(obj: dict) -> str:
    """
    gets the method from the yaml object
    """
    if obj.get("generate_background"):
        return "RAG+background"
    additional = obj.get("additional_collections", [])
    if additional:
        assert len(additional) == 1
        x = additional[0]
        if x.startswith("gh_"):
            return "RAG+github"
        elif x.startswith("devdocs"):
            return "RAG+devdocs"
        else:
            assert False
    else:
        return "RAG"

from pathlib import Path
def load_definition_results():
    files = glob.glob("results/*-Pdefinition-*.results.tsv")
    dfs = []
    for file in files:
        yaml_path = file.replace(".tsv", ".yaml")
        if not Path(yaml_path).exists():
            continue
        meta = yaml.safe_load(open(yaml_path))
        res = pd.read_csv(file, comment="#", sep="\t")
        res['task'] =  " ".join(meta.get("fields_to_predict"))
        res['ontology'] = meta.get("source_collection", "").replace("ont_", "")
        res["method"] = get_method(meta)
        res["model_name"] = meta["model_name"]
        dfs.append(res)
    return pd.concat(dfs)
        
df = load_definition_results()

In [4]:
df

,masked_id,feature_label,expected_definition,feature_aliases,feature_relationships,feature_logical_definition,masked_original_id,predicted_definition,metric_precision,metric_recall,...,ontology,method,model_name,id,label,definition,aliases,relationships,logical_definition,original_id
0,PignutHickoryNut,pignut hickory nut,A whole hickory nut from a pignut hickory tree...,NaN,"[{'predicate': 'InTaxon', 'target': 'CaryaGlab...","[{'predicate': 'rdfs:subClassOf', 'target': 'C...",FOODON:00004448,A caryopsis fruit of a pignut hickory tree (Ca...,0.0,0.0,...,foodon,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,GreekYogurt,greek yogurt,NaN,NaN,"[{'predicate': 'subClassOf', 'target': 'Strain...",NaN,FOODON:00004409,Greek yogurt is a type of yogurt that is made ...,0.0,0.0,...,foodon,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TurkeyMeat_ground_sauteed_,"turkey meat (ground, sauteed)",Ground turkey meat which has been sauteed.,NaN,"[{'predicate': 'subClassOf', 'target': 'Turkey...",NaN,FOODON:00004417,Ground sautéed turkey meat is a specific type ...,0.0,0.0,...,foodon,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Raw,raw,"A characteristic of an animal, plant, fungi or...",NaN,"[{'predicate': 'subClassOf', 'target': 'FoodOb...",NaN,FOODON:00004348,NaN,0.0,0.0,...,foodon,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SpeltFoodProduct,spelt food product,A food product made from spelt wheat.,NaN,"[{'predicate': 'subClassOf', 'target': 'WheatF...",NaN,FOODON:00004434,NaN,0.0,0.0,...,foodon,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,DrySoilAlbedo,dry soil albedo,Soil albedo which inheres in dry soil.,NaN,"[{'predicate': 'subClassOf', 'target': 'SoilAl...",NaN,ENVO:06105233,Soil albedo that specifically focuses on the r...,0.0,0.0,...,envo,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,PediatricIntensiveCareUnitFacility,pediatric intensive care unit facility,A hospital unit facility which is used to prov...,NaN,"[{'predicate': 'subClassOf', 'target': 'Intens...",NaN,ENVO:03501170,A specialized healthcare facility that provide...,0.0,0.0,...,envo,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,NestOfAlligator,nest of alligator,An animal habitation which is built by alligat...,NaN,"[{'predicate': 'subClassOf', 'target': 'Animal...",NaN,ENVO:03600069,A specific physical structure or location wher...,0.0,0.0,...,envo,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48,ConcentrationOfCarbon13AtomInSeaWater,concentration of carbon-13 atom in sea water,The concentration of carbon-13 atom when measu...,NaN,"[{'predicate': 'subClassOf', 'target': 'Concen...","[{'predicate': 'rdfs:subClassOf', 'target': 'C...",ENVO:3100042,The concentration of carbon-13 atom when measu...,1.0,1.0,...,envo,RAG+background,gpt-3.5-turbo,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
# count number of rows grouped by model_name and method
df.groupby(["model_name", "method"]).size()

model_name       method        
gpt-3.5-turbo    RAG               490
                 RAG+background    440
                 RAG+devdocs        50
                 RAG+github        190
gpt-4            RAG               534
                 RAG+background    390
                 RAG+github        190
nous-hermes-13b  RAG               490
                 RAG+background     40
                 RAG+github         40
dtype: int64

In [5]:
from bert_score import score as bscore

In [43]:
def get_bert_score(df, model_name, method, n=None):
    df2 = df[(df.model_name == model_name) & (df.method == method)]
    df2 = df2.dropna(subset=["expected_definition", "predicted_definition"])
    refs = df2["expected_definition"].values
    hyps = df2["predicted_definition"].values
    # convert np array to list
    refs = [x for x in refs]
    hyps = [x for x in hyps]
    if n:
        refs = refs[:n]
        hyps = hyps[:n]
    P, R, F1 = bscore(hyps, refs, lang="en", verbose=False)
    return P.mean().tolist(), R.mean().tolist(), F1.mean().tolist()

#get_bert_score(df, "gpt-3.5-turbo", "RAG")

In [44]:
rows = []
for model in MODELS:
    for method in METHODS:
        if method == "RAG+github" and model == "nous-hermes-13b":
            continue
        avg_p, avg_r, avg_f1 = get_bert_score(df, model, method)
        row = {"model": model, "method": method, "F1": avg_f1, "P": avg_p, "R": avg_r}
        rows.append(row)
        print(row)
        

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'gpt-3.5-turbo', 'method': 'RAG', 'F1': 0.9232134819030762, 'P': 0.9330469965934753, 'R': 0.9142860174179077}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'gpt-3.5-turbo', 'method': 'RAG+github', 'F1': 0.9242426753044128, 'P': 0.9273239374160767, 'R': 0.9219368100166321}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'gpt-4', 'method': 'RAG', 'F1': 0.9199519753456116, 'P': 0.9283441305160522, 'R': 0.9123955965042114}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'gpt-4', 'method': 'RAG+github', 'F1': 0.9257653951644897, 'P': 0.9291232824325562, 'R': 0.9231078624725342}


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'model': 'nous-hermes-13b', 'method': 'RAG', 'F1': 0.9158900380134583, 'P': 0.9220759868621826, 'R': 0.9104764461517334}


In [45]:
len(rows)

5

In [46]:
results_df = pd.DataFrame(rows)
results_df

,model,method,F1,P,R
0,gpt-3.5-turbo,RAG,0.923213,0.933047,0.914286
1,gpt-3.5-turbo,RAG+github,0.924243,0.927324,0.921937
2,gpt-4,RAG,0.919952,0.928344,0.912396
3,gpt-4,RAG+github,0.925765,0.929123,0.923108
4,nous-hermes-13b,RAG,0.915890,0.922076,0.910476


In [49]:
# filter and show with precision=3
results_df[results_df.method == "RAG"].round(3)

,model,method,F1,P,R
0,gpt-3.5-turbo,RAG,0.923,0.933,0.914
2,gpt-4,RAG,0.920,0.928,0.912
4,nous-hermes-13b,RAG,0.916,0.922,0.910


In [85]:
df2 = df.dropna(subset=["expected_definition", "predicted_definition"])
random_defs = df2["expected_definition"].values
random_defs = [x for x in random_defs]

# randomize order
import random
random.seed(42)
random.shuffle(random_defs)
all_refs = df2["expected_definition"].values

all_refs = [x for x in all_refs]


In [86]:
t = bscore(random_defs[:10], all_refs[:10], lang="en", verbose=False)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [89]:
t[0].mean(), t[1].mean(), t[2].mean()

(tensor(0.8452), tensor(0.8320), tensor(0.8384))

In [90]:
t[0].mean(), t[1].mean(), t[2].mean()


(tensor(0.8452), tensor(0.8320), tensor(0.8384))

In [76]:
all_preds[1], all_refs[1]

('An alpha-beta CD4 T cell that resides in the lung. This cell expresses the resident marker CD69 and has the effector/memory phenotype, high CD44 (hyaluronic acid receptor) expression and low CD62L expression. A lung resident memory CD4-positive, alpha-beta T cell upregulates specific genes associated with tissue residency and downregulates genes associated with circulation and migration.',
 'The amount of a IgA immunoglobulin complex, circulating when measured in blood serum.')